In [1]:
import json, time
from websocket import create_connection

In [2]:
import json
import websocket
import pandas as pd

# Initialize an empty DataFrame to store the trade data
df = pd.DataFrame(columns=["time", "price", "volume"])

def on_message(ws, message):
    message = json.loads(message)

    if isinstance(message, dict):
        event = message.get("event")
        if event == "heartbeat":
            return  # Skip processing if the event is a heartbeat
        else:
            print(message)
    elif isinstance(message, list):
        print(message)
        return 
        # Extract the trade information from the message
        trade = message["XBT/USD"]
        # Append the trade information to the DataFrame
        df.loc[len(df)] = [trade[2], trade[0], trade[1]]
        print(df)

def on_error(ws, error):
    print(error)

def on_close(ws):
    print("Closed connection")

def on_open(ws):
    ws.send(json.dumps({
        "event": "subscribe",
        "pair": ["XBT/USD"],
        "subscription": {"name": "trade"},
        #"subscription": {"name": "book", "depth": 10}
    }))

if __name__ == "__main__":
    websocket.enableTrace(True)
    ws = websocket.WebSocketApp("wss://ws.kraken.com",
                              on_message = on_message,
                              on_error = on_error,
                              on_close = on_close)
    ws.on_open = on_open
    ws.run_forever()


--- request header ---
GET / HTTP/1.1
Upgrade: websocket
Connection: Upgrade
Host: ws.kraken.com
Origin: http://ws.kraken.com
Sec-WebSocket-Key: +s8Cwhmr4Wl2BRoXw1p9Fg==
Sec-WebSocket-Version: 13


-----------------------
--- response header ---
HTTP/1.1 101 Switching Protocols
Date: Fri, 20 Jan 2023 23:25:05 GMT
Connection: upgrade
Set-Cookie: AWSALB=FIUV++4GU2P1Kbkr378dR++UXLEKChY1Hhk5hJJNCTH13l4qxck+st15rsCsMuBqpclpol7CE8PVO5hzuTpXRbytFm/kcNfU1XLr28iWZFUBmHtuXiVAKQAKnE7Y; Expires=Fri, 27 Jan 2023 23:25:05 GMT; Path=/
Upgrade: websocket
Sec-WebSocket-Accept: zIVt4YK2B/qXwRiKWFBlp/yqnns=
CF-Cache-Status: DYNAMIC
Strict-Transport-Security: max-age=31536000; includeSubDomains; preload
X-Content-Type-Options: nosniff
Set-Cookie: AWSALBCORS=FIUV++4GU2P1Kbkr378dR++UXLEKChY1Hhk5hJJNCTH13l4qxck+st15rsCsMuBqpclpol7CE8PVO5hzuTpXRbytFm/kcNfU1XLr28iWZFUBmHtuXiVAKQAKnE7Y; Expires=Fri, 27 Jan 2023 23:25:05 GMT; Path=/; SameSite=None; Secure
Set-Cookie: __cf_bm=NUfWjSBEFVfcUMAfsL.qWWcvO46qEgmDdcmLU

{'connectionID': 9187197382675103737, 'event': 'systemStatus', 'status': 'online', 'version': '1.9.0'}
{'errorMessage': "Unsupported field: 'channels' for the given msg type: subscribe", 'event': 'subscriptionStatus', 'pair': 'XBT/USD', 'status': 'error', 'subscription': {'name': 'trade'}}


send: b'\x88\x82\x80\xf7\xb8\x84\x83\x1f'



Closed connection
